In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
from food.tools import *
from food.paths import *
from food.psql import *

import pandas as pd
import numpy as np

# !nbdev_build_lib
import  plotly.express as px

In [5]:
pd.read_sql('select * from dishes',engine)

,id,food_id,description,category,energy,protein,carb,fat,score,image_url,user_id,grams,measure_selected,timestamp
0,15,1100253,"Egg omelet or scrambled egg, with meat, made w...",Eggs and omelets,162.0,13.49,1.00,11.16,0.272141,https://api.telegram.org/file/bot5091011572:AA...,78882798,70,1 egg,2022-04-05 12:02:25.527475+00:00
1,16,1100191,"Egg, whole, fried with animal fat or meat drip...",Eggs and omelets,188.0,11.85,0.67,14.88,0.253294,https://api.telegram.org/file/bot5091011572:AA...,78882798,55,1 egg,2022-04-05 12:02:47.291946+00:00
2,17,1100839,"Biscuit, wheat","Biscuits, muffins, quick breads",362.0,7.50,43.90,18.19,0.265885,https://api.telegram.org/file/bot5091011572:AA...,78882798,45,1 biscuit,2022-04-05 12:03:39.342587+00:00


In [6]:
pd.read_sql('select * from foods where id = 1100839',engine)

,id,description,category,energy,protein,carb,fat,clip
0,1100839,"Biscuit, wheat","Biscuits, muffins, quick breads",362.0,7.5,43.9,18.19,"[-0.016676111, 0.012137895, 0.020485425, -0.00..."


take single shot to assess nutrition value of the dish without a need to weight (separately or entirely), search or input


image multiclassificator
- classes
	- limited  number of classes, multiplied by  grid of weights. Something  simple, like "Grilled salmon with  white sauce" * [50 g, 100 g, 150 g, 200 g, 250 g, 300 g]
- calculation
	- KNN search for the image around text vectors. Next, take farthest  neighbours from KNN to pick the different types of food and remove synonyms  
- output
	- something like [Grilled salmon with white sauce 150 g, green salad 100 g]

1 test if weight would work
- to database
- select a few difference types of food, inference weighted weighted
- check accuracy

In [6]:
from IPython.display import display

In [7]:
data_path.ls()

[PosixPath('/home/dima/data/full'),
 PosixPath('/home/dima/data/branded'),
 PosixPath('/home/dima/data/FoundationFoods'),
 PosixPath('/home/dima/data/FNDDS'),
 PosixPath('/home/dima/data/support_data')]

In [8]:
fundation_path = data_path/'FoundationFoods'
fndds_path = data_path/'FNDDS'
support_path = data_path/'support_data'
branded_path = data_path/'branded'
full_path = data_path/'full'


paths = [fundation_path,fndds_path,support_path,branded_path,full_path]

In [2]:
def display_df_heads(path,min_len = 0):
    for f in [f for f in path.ls() if '.csv' in f.name]:
        print(f)
        df = pd.read_csv(f)
        if len(df)>=min_len:
            display(df.head())
            print(len(df))

In [18]:
display_df_heads(support_path)

/home/dima/data/support_data/food_nutrient_derivation.csv


,id,code,description,source_id
0,1,A,Analytical,1
1,2,AI,Analytical data; from the literature or gover...,10
2,3,AR,Analytical data; derived by linear regression,1
3,4,AS,Summed,1
4,5,BD,Based on same food; Drained solids from solids...,2


64
/home/dima/data/support_data/fndds_derivation.csv


,SR 28 derivation code,SR 28 derivation description
0,A,Analytical
1,AI,Analytical data; from the literature or gover...
2,AR,Analytical data; derived by linear regression
3,AS,Summed
4,BD,Based on same food; Drained solids from solids...


49
/home/dima/data/support_data/nutrient.csv


...

122
/home/dima/data/support_data/lab_method.csv


,id,description,technique
0,1000,NIST Handbook 133,Gravimetric
1,1001,AOAC 968.06 + 992.15,Combustion
2,1002,AOAC 960.39 39.1,Extraction
3,1003,AOAC 922.06,Acid hydrolysis
4,1004,AOAC 923.03,Gravimetric


274
/home/dima/data/support_data/lab_method_code.csv


,id,lab_method_id,code
0,1000,1000,SPGP_S
1,1001,1001,DGEN_S
2,1003,1003,FAT_AH_S
3,1004,1004,ASHM_S
4,1005,1005,SUGN_S


292


In [11]:
display_df_heads(fndds_path)


/home/dima/data/FNDDS/survey_fndds_food.csv


[autoreload of food.psql failed: Traceback (most recent call last):
  File "/home/dima/anaconda3/envs/food/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 312, in update_class
    new_obj = getattr(new, key)
AttributeError: type object 'Foods' has no attribute 'category'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/dima/anaconda3/envs/food/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 257, in check
    superreload(m, reload, self.old_objects)
  File "/home/dima/anaconda3/envs/food/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 480, in superreload
    update_generic(old_obj, new_obj)
  File "/home/dima/anaconda3/envs/food/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 377, in update_generic
    update(a, b)
  File "/home/dima/anaconda3/envs/food/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 320, in update_clas

,fdc_id,food_code,wweia_category_number,start_date,end_date
0,1097510,11000000,9602,2017-01-01,2018-12-31
1,1097511,11100000,1004,2017-01-01,2018-12-31
2,1097512,11111000,1002,2017-01-01,2018-12-31
3,1097513,11111100,1002,2017-01-01,2018-12-31
4,1097514,11111150,1002,2017-01-01,2018-12-31


7083
/home/dima/data/FNDDS/food_update_log_entry.csv


,id,description,last_updated
0,1097510,"Milk, human",2020-10-30
1,1097511,"Milk, NFS",2020-10-30
2,1097512,"Milk, whole",2020-10-30
3,1097513,"Milk, low sodium, whole",2020-10-30
4,1097514,"Milk, calcium fortified, whole",2020-10-30


...

24896
/home/dima/data/FNDDS/food.csv


,fdc_id,data_type,description,food_category_id,publication_date
0,1097510,survey_fndds_food,"Milk, human",NaN,2020-10-30
1,1097511,survey_fndds_food,"Milk, NFS",NaN,2020-10-30
2,1097512,survey_fndds_food,"Milk, whole",NaN,2020-10-30
3,1097513,survey_fndds_food,"Milk, low sodium, whole",NaN,2020-10-30
4,1097514,survey_fndds_food,"Milk, calcium fortified, whole",NaN,2020-10-30


7083
/home/dima/data/FNDDS/food_nutrient.csv


,id,fdc_id,nutrient_id,amount,data_points,derivation_id,min,max,median,sf.footnote,min_year_acquired
0,12875253,1097510,1180,16.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,12875267,1097510,1264,0.321,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,12875281,1097510,1292,1.658,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,12875282,1097510,1293,0.497,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,12875262,1097510,1259,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN


460395


In [99]:
food = food[['nutrient bank number','category','description','kilocalories','carbohydrate','protein','fat.total lipid']]

In [100]:
food = food.rename(columns = {'fat.total lipid':'fat','nutrient bank number':'id'})

In [104]:
food[food['description'].str.contains('chocolate')]

,id,category,description,kilocalories,carbohydrate,protein,fat
1914,8204,cereals rte,"cereals rte,chocolate-flavored frstd puffed corn",405,87.20,3.34,3.50
2160,8547,cereals rte,"cereals rte,quaker,pnut butter chocolatey crun...",415,79.40,7.37,8.65
4126,14175,chocolate-flavor bev mix for milk,"chocolate-flavor bev mix for milk,pdr,wo/ adde...",405,90.90,3.30,3.10
4127,14177,chocolate-flavor bev mix,"chocolate-flavor bev mix,pdr,prep w/ whl milk",85,11.91,3.23,3.24
4128,14181,no category,chocolate syrup,279,65.10,2.10,1.13
4129,14182,chocolate syrup,"chocolate syrup,prep w/ whl milk",90,12.78,3.07,2.96
4190,14346,shake,"shake,fast food,chocolate",127,20.50,3.40,3.70
4265,14552,carbonated bev,"carbonated bev,chocolate-flavored soda",42,10.70,0.00,0.00
4268,14557,chocolate-flavor bev mix for milk,"chocolate-flavor bev mix for milk,pdr,w/ added...",400,90.28,4.55,2.27
5310,18127,cake,"cake, snack cakes, creme-filled, chocolate wit...",399,60.31,3.63,15.93


In [93]:
food.to_sql('foods',engine,index=False,if_exists='append')

In [5]:
pdrows(200)

In [1]:
df

NameError: name 'df' is not defined

In [16]:
df.columns.str.replace('Data.','').str.replace('Data.','')

/tmp/ipykernel_1871201/4192578409.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df.columns.str.replace('Data.','')


Index(['Category', 'Description', 'Nutrient Bank Number', 'Alpha Carotene',
       'Ash', 'Beta Carotene', 'Beta Cryptoxanthin', 'Carbohydrate',
       'Cholesterol', 'Choline', 'Fiber', 'Kilocalories',
       'Lutein and Zeaxanthin', 'Lycopene', 'Manganese', 'Niacin',
       'Pantothenic Acid', 'Protein', 'Refuse Percentage', 'Retinol',
       'Riboflavin', 'Selenium', 'Sugar Total', 'Thiamin', 'Water',
       'Fat.Monosaturated Fat', 'Fat.Polysaturated Fat', 'Fat.Saturated Fat',
       'Fat.Total Lipid', 'Household Weights.1st Household Weight',
       'Household Weights.1st Household Weight Description',
       'Household Weights.2nd Household Weight',
       'Household Weights.2nd Household Weight Description',
       'Major Minerals.Calcium', 'Major Minerals.Copper',
       'Major Minerals.Iron', 'Major Minerals.Magnesium',
       'Major Minerals.Phosphorus', 'Major Minerals.Potassium',
       'Major Minerals.Sodium', 'Major Minerals.Zinc',
       'Vitamins.Vitamin A - IU', 'Vitam

In [ ]:
dataset - should give 

In [85]:
pdrows(50)

In [89]:
df[df['Category'].str.contains('FRUIT')]

,Category,Description,Nutrient Data Bank Number,Data.Alpha Carotene,Data.Ash,Data.Beta Carotene,Data.Beta Cryptoxanthin,Data.Carbohydrate,Data.Cholesterol,Data.Choline,...,Data.Major Minerals.Potassium,Data.Major Minerals.Sodium,Data.Major Minerals.Zinc,Data.Vitamins.Vitamin A - IU,Data.Vitamins.Vitamin A - RAE,Data.Vitamins.Vitamin B12,Data.Vitamins.Vitamin B6,Data.Vitamins.Vitamin C,Data.Vitamins.Vitamin E,Data.Vitamins.Vitamin K
2236,BREADFRUIT,"breadfruit,raw",9059,0,0.93,0,0,27.12,0,10,...,490,2,0.12,0,0,0.0,0.100,29.0,0.10,0.5
2271,FRUIT COCKTAIL,"fruit cocktail,cnd,h2o pk,sol&liquids",9096,0,0.26,126,48,8.51,0,4,...,94,4,0.09,250,13,0.0,0.052,2.1,0.40,2.6
2272,FRUIT COCKTAIL,"fruit cocktail,cnd,juc pk,sol&liquids",9097,0,0.23,154,58,11.86,0,4,...,95,4,0.09,305,15,0.0,0.051,2.7,0.40,2.6
2273,FRUIT COCKTAIL,"fruit cocktail,cnd,ex lt syrup,sol&liquids",9098,0,0.20,0,0,11.63,0,0,...,104,4,0.08,233,12,0.0,0.051,3.0,0.00,0.0
2274,FRUIT COCKTAIL,"fruit cocktail,cnd,lt syrup,sol&liquids",9099,0,0.23,104,39,14.93,0,4,...,89,6,0.09,208,10,0.0,0.051,1.9,0.50,2.6
2275,FRUIT COCKTAIL,"fruit cocktail,cnd,hvy syrup,sol&liquids",9100,0,0.23,104,39,18.91,0,4,...,88,6,0.08,205,10,0.0,0.050,1.9,0.40,2.6
2276,FRUIT COCKTAIL,"fruit cocktail,cnd,ex hvy syrup,sol&liquids",9101,0,0.22,0,0,22.89,0,0,...,86,6,0.08,201,10,0.0,0.049,1.9,0.00,0.0
2277,FRUIT SALAD,"fruit salad,cnd,h2o pk,sol&liquids",9102,0,0.22,0,0,7.87,0,0,...,78,3,0.08,440,22,0.0,0.032,1.9,0.00,0.0
2278,FRUIT SALAD,"fruit salad,cnd,juc pk,sol&liquids",9103,0,0.26,0,0,13.05,0,0,...,116,5,0.14,600,30,0.0,0.027,3.3,0.00,0.0
2279,FRUIT SALAD,"fruit salad,cnd,lt syrup,sol&liquids",9104,0,0.21,0,0,15.14,0,0,...,82,6,0.07,429,21,0.0,0.032,2.5,0.00,0.0


In [86]:
df.groupby(['Category']).count().sort_values('Description',ascending =False).head(50)

,Description,Nutrient Data Bank Number,Data.Alpha Carotene,Data.Ash,Data.Beta Carotene,Data.Beta Cryptoxanthin,Data.Carbohydrate,Data.Cholesterol,Data.Choline,Data.Fiber,...,Data.Major Minerals.Potassium,Data.Major Minerals.Sodium,Data.Major Minerals.Zinc,Data.Vitamins.Vitamin A - IU,Data.Vitamins.Vitamin A - RAE,Data.Vitamins.Vitamin B12,Data.Vitamins.Vitamin B6,Data.Vitamins.Vitamin C,Data.Vitamins.Vitamin E,Data.Vitamins.Vitamin K
Category,,,,,,,,,,,,,,,,,,,,,
BEEF,457,457,457,457,457,457,457,457,457,457,...,457,457,457,457,457,457,457,457,457,457
PORK,304,304,304,304,304,304,304,304,304,304,...,304,304,304,304,304,304,304,304,304,304
CEREALS RTE,291,291,291,291,291,291,291,291,291,291,...,291,291,291,291,291,291,291,291,291,291
BABYFOOD,221,221,221,221,221,221,221,221,221,221,...,221,221,221,221,221,221,221,221,221,221
No Category,198,198,198,198,198,198,198,198,198,198,...,198,198,198,198,198,198,198,198,198,198
LAMB,198,198,198,198,198,198,198,198,198,198,...,198,198,198,198,198,198,198,198,198,198
SOUP,188,188,188,188,188,188,188,188,188,188,...,188,188,188,188,188,188,188,188,188,188
CHICKEN,163,163,163,163,163,163,163,163,163,163,...,163,163,163,163,163,163,163,163,163,163
CEREALS,132,132,132,132,132,132,132,132,132,132,...,132,132,132,132,132,132,132,132,132,132


In [84]:
df[df['Category']=='No Category']

,Category,Description,Nutrient Data Bank Number,Data.Alpha Carotene,Data.Ash,Data.Beta Carotene,Data.Beta Cryptoxanthin,Data.Carbohydrate,Data.Cholesterol,Data.Choline,...,Data.Major Minerals.Potassium,Data.Major Minerals.Sodium,Data.Major Minerals.Zinc,Data.Vitamins.Vitamin A - IU,Data.Vitamins.Vitamin A - RAE,Data.Vitamins.Vitamin B12,Data.Vitamins.Vitamin B6,Data.Vitamins.Vitamin C,Data.Vitamins.Vitamin E,Data.Vitamins.Vitamin K
55,No Category,eggnog,1057,0,0.80,6,1,13.54,59,52,...,165,54,0.46,161,46,0.45,0.050,1.5,0.20,0.2
149,No Category,cheese fondue,1163,0,1.85,0,0,3.77,45,0,...,105,132,1.96,414,109,0.83,0.055,0.0,0.00,0.0
167,No Category,kraft cheez whiz past process chs sau,1188,0,5.80,0,0,9.20,75,0,...,240,1638,1.64,649,0,0.00,0.000,0.4,0.00,0.0
168,No Category,kraft cheez whiz lt past process chs product,1189,0,6.40,0,0,16.20,35,0,...,297,1705,2.36,628,0,0.00,0.000,0.4,0.00,0.0
169,No Category,kraft free singles american nonfat past proces...,1190,0,6.10,0,0,11.70,16,0,...,236,1298,2.50,2166,0,0.00,0.000,0.2,0.00,0.0
170,No Category,kraft velveeta past process chs sprd,1191,0,6.00,0,0,9.80,80,0,...,335,1499,1.84,1107,0,0.00,0.000,0.2,0.00,0.0
171,No Category,kraft velveeta lt red fat past process chs pro...,1192,0,6.60,0,0,11.80,42,0,...,345,1586,2.49,982,0,0.00,0.000,0.1,0.00,0.0
172,No Category,kraft breakstone's red fat sour crm,1193,0,0.80,0,0,6.50,50,0,...,210,59,0.00,1053,0,0.00,0.000,1.1,0.00,0.0
173,No Category,kraft breakstone's free fat free sour crm,1194,0,1.20,0,0,15.10,9,0,...,219,72,0.00,679,0,0.00,0.000,1.2,0.00,0.0
174,No Category,kraft breyers lowfat strawberry yogurt (1% mil...,1195,0,0.70,0,0,18.20,9,0,...,192,52,0.00,34,0,0.53,0.000,0.0,0.00,0.0


In [26]:
df['Description'] = df['Description'].str.lower()

In [27]:
[c for c in df.columns if 'Household' in c]

['Data.Household Weights.1st Household Weight',
 'Data.Household Weights.1st Household Weight Description',
 'Data.Household Weights.2nd Household Weight',
 'Data.Household Weights.2nd Household Weight Description']

In [94]:
apples = df[df['Description'].str.contains('')&df['Category'].str.contains('APPLES')][['Description']+[c for c in df.columns if 'Weight' in c]]

In [95]:
apples

,Description,Data.Household Weights.1st Household Weight,Data.Household Weights.1st Household Weight Description,Data.Household Weights.2nd Household Weight,Data.Household Weights.2nd Household Weight Description
2186,"apples,raw,with skin",125.00,"1 cup, quartered or chopped",109,"1 cup, slices"
2187,"apples,raw,without skin",110.00,"1 cup, slices",216,"1 large, (3-1/4"" dia)"
2188,"apples,raw,wo/skn,ckd,bld",171.00,"1 cup, slices",0,NaN
2189,"apples,raw,wo/ skn,ckd,microwave",170.00,"1 cup, slices",0,NaN
2190,"apples,cnd,swtnd,sliced,drnd,unhtd",204.00,"1 cup, slices",0,NaN
2191,"apples,cnd,swtnd,sliced,drnd,htd",204.00,"1 cup, slices",0,NaN
2192,"apples,dehyd (lo moist),sulfured,unckd",60.00,1 cup,0,NaN
2193,"apples,dehyd (lo moist),sulfured,stwd",193.00,1 cup,0,NaN
2194,"apples,dried,sulfured,unckd",86.00,1 cup,6,1 ring
2195,"apples,dried,sulfured,stwd,wo/ sugar",255.00,1 cup,0,NaN


In [69]:
apples.groupby("Data.Household Weights.1st Household Weight Description")['Description'].count()

Series([], Name: Description, dtype: int64)

In [45]:
distr = df.groupby(['Data.Household Weights.1st Household Weight Description'])['Description'].count().sort_values(ascending=False)

In [58]:
distr = distr[distr>20]

In [59]:
px.line(distr)

In [39]:
df[df['Data.Household Weights.1st Household Weight Description'] == '1 serving']

,Category,Description,Nutrient Data Bank Number,Data.Alpha Carotene,Data.Ash,Data.Beta Carotene,Data.Beta Cryptoxanthin,Data.Carbohydrate,Data.Cholesterol,Data.Choline,...,Data.Major Minerals.Potassium,Data.Major Minerals.Sodium,Data.Major Minerals.Zinc,Data.Vitamins.Vitamin A - IU,Data.Vitamins.Vitamin A - RAE,Data.Vitamins.Vitamin B12,Data.Vitamins.Vitamin B6,Data.Vitamins.Vitamin C,Data.Vitamins.Vitamin E,Data.Vitamins.Vitamin K
415,BABYFOOD,"babyfood,juc,appl,w/ca",3269,0,0.48,0,0,11.10,0,2,...,92,3,0.03,0,0,0.00,0.030,21.2,0.00,0.0
1020,CHICKEN,"chicken,wing,frz,glazed,barbecue flav,htd (mic...",5313,0,3.04,0,0,3.84,156,0,...,254,837,1.55,68,20,0.48,0.210,0.5,0.38,0.0
1025,CHICKEN,"chicken,wing,frz,glazed,bbq flav,htd (conventi...",5320,0,2.28,0,0,3.36,136,0,...,218,559,1.30,69,21,0.55,0.183,0.0,0.44,0.0
1146,SOUP,"soup,pea,grn,cnd,cond,prep w/ eq volume h2o",6069,0,1.28,1,0,9.88,0,13,...,71,344,0.64,12,3,0.00,0.020,0.6,0.09,0.2
1175,GRAVY,"gravy,pork,dry,pdr",6124,0,14.59,2,0,63.57,10,24,...,235,5356,1.06,117,34,0.51,0.086,1.4,0.26,0.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6998,CAMPBELL SOUP COMPANY,"campbell soup company,pace,triple pepper salsa",27030,0,1.30,0,0,9.38,0,0,...,0,594,0.00,625,0,0.00,0.000,7.5,0.00,0.0
7009,No Category,campbell's v8 100% veg juc,31008,0,1.16,0,0,4.12,0,0,...,193,198,0.00,823,0,0.00,0.000,29.6,0.00,0.0
7399,PUDDINGS,"puddings,choc flavor,lo cal,inst,dry mix",44061,0,9.90,0,0,78.20,0,3,...,1279,2838,0.41,0,0,0.01,0.040,0.0,0.02,0.4
7403,PUDDINGS,"puddings,choc flavor,lo cal,reg,dry mix",44258,0,5.70,0,0,74.42,0,3,...,570,3326,1.49,0,0,0.00,0.027,0.0,0.02,0.5


- test if will better work in less text is used. for banana
1 select all where 'banan' is present
2 inference for only 'product name' short
3 create milvus collection for short and long texts and play with

In [3]:
pdrows(200)

In [7]:
paths = data_path.ls()

In [14]:
n=1

In [21]:
pdrows(10)

In [37]:
p = paths[n]
print(n)
print(p)
n+=1
pd.read_csv(p)

IndexError: list index out of range

In [ ]:
pd.read_sql(data_path)

In [3]:
def get_column_names(Table):
    cols = list(Table.__table__.c)
    return [c.name for c in cols]

create table for babanas

In [4]:
cols = get_column_names(Bananas)
cols = [c for c in cols if not 'clip' in c]

In [5]:
cols

['id',
 'product_name',
 'keywords',
 'ingredients_text',
 'categories',
 'food_groups',
 'energy_kcal_100g',
 'proteins_100g',
 'fat_100g',
 'carbohydrates_100g']

In [6]:
df = pd.read_sql(f'select {",".join(cols)} from foods_big',engine)

In [8]:
df['food_groups'].unique()

array(['ice-cream', 'NaN', 'dressings-and-sauces', 'bread',
       'one-dish-meals', 'meat-other-than-poultry', 'processed-meat',
       'chocolate-products', 'milk-and-yogurt', 'cheese',
       'dairy-desserts', 'unknown', 'fruit-juices', 'eggs', 'legumes',
       'biscuits-and-cakes', 'breakfast-cereals', 'fats', 'sweets',
       'fruits', 'sweetened-beverages', 'vegetables',
       'alcoholic-beverages', 'artificially-sweetened-beverages',
       'lean-fish', 'fruit-nectars', 'salty-and-fatty-products',
       'pizza-pies-and-quiches', 'cereals', 'sandwiches',
       'plant-based-milk-substitutes', 'nuts', 'appetizers',
       'fish-and-seafood', 'dried-fruits', 'soups', 'poultry',
       'teas-and-herbal-teas-and-coffees', 'unsweetened-beverages',
       'offals', 'fruits-and-vegetables', 'pastries', 'potatoes',
       'cereals-and-potatoes', 'waters-and-flavored-waters', 'fatty-fish',
       'fish-meat-eggs'], dtype=object)

In [15]:
df.groupby('food_groups')['product_name'].count().sort_values(ascending=False)

food_groups
NaN                                 683748
biscuits-and-cakes                   44884
sweets                               42842
dressings-and-sauces                 29338
one-dish-meals                       26978
cereals                              26235
cheese                               23317
processed-meat                       19504
milk-and-yogurt                      19103
sweetened-beverages                  16006
fish-and-seafood                     15329
bread                                15170
vegetables                           13849
unsweetened-beverages                13404
chocolate-products                   10489
fats                                  9981
appetizers                            8251
meat-other-than-poultry               7843
fruits                                7753
salty-and-fatty-products              7392
breakfast-cereals                     6909
poultry                               6158
pizza-pies-and-quiches                5993

In [48]:
def bing_search_image(q):
    url = "https://bing-image-search1.p.rapidapi.com/images/search"

    querystring = {"q":q,"count":"1"}

    headers = {
        "X-RapidAPI-Host": "bing-image-search1.p.rapidapi.com",
        "X-RapidAPI-Key": "BI6oP0xGxsmsh6qLhuW9H4zO9gKcp149uMujsnNNKgjMVUdAxg"
    }

    r = requests.request("GET", url, headers=headers, params=querystring)
    return [el['contentUrl'] for el in json.loads(r.text)['value']]

In [50]:
df[df['food_groups']=='vegetables'].sample(10)

,id,product_name,keywords,ingredients_text,categories,food_groups,energy_kcal_100g,proteins_100g,fat_100g,carbohydrates_100g
454053,672372,Tomate Frito Ecologico,"su, verdura, alimento, tomatoes-and-tomato-pro...",NaN,"Alimentos y bebidas de origen vegetal, Aliment...",vegetables,76.000000,1.500000,3.100000,10.000000
606165,936844,Kelloggs original sausage patties clean eating...,"pattie, and, eating, analogue, sausage, frozen...","advice. Ingredie.nts: Water, wheat gluten, soy...","Plant-based foods and beverages, Plant-based f...",vegetables,210.526316,23.684211,7.894737,13.157895
120358,169267,"Glaser Organic Farms, Cabbage & Jalapeno Sauer...","based, beverage, vegetable, and, plant-based, ...","Organic green cabbage, organic jalapeno, celti...","Plant-based foods and beverages, Plant-based f...",vegetables,42.000000,2.820000,0.000000,8.450000
361623,532840,Macedoine De Legumes Extra 37CL,"base, macedoine, de, canned-drained-diced-mixe...",NaN,"Aliments et boissons à base de végétaux, Alime...",vegetables,159.000000,2.100000,0.300000,5.000000
728464,3546,Chopped Mixed Greens,"beverage, green, based, mixed, kroger, fruit, ...","Turnip greens, mustard greens, water, salt.","Plant-based foods and beverages, Plant-based f...",vegetables,13.000000,0.870000,0.000000,2.610000
151858,214708,Tourte champignons jambon,"verte, plante, chine, tige, base, tourte, cham...","Asperges vertes, eau, sel, acidifiant : acide ...","Aliments et boissons à base de végétaux, Alime...",vegetables,248.000000,4.600000,16.000000,22.000000
136633,189744,Mesclun (5/6 portions),"base, 5-6, san, portion, florette, frai, verte...",Mesclun (mélange de jeunes pousses comprenant ...,"Aliments et boissons à base de végétaux, Alime...",vegetables,18.000000,2.200000,0.500000,0.500000
225105,324656,Betteraves Rouges Entières,"origine, entiere, base, fruit, betterave, de, ...",,"Aliments et boissons à base de végétaux, Alime...",vegetables,52.000000,0.900000,0.400000,11.000000
39352,59364,Young sweet corn in brine,"coconut, corn, based, in, co, beverage, food, ...","Young sweet corn, water, salt, citric acid (as...","Plant-based foods and beverages, Plant-based f...",vegetables,25.000000,1.670000,0.000000,5.000000
155629,221229,salsifis,"vegetaux, origine, made-in-france, et, en, con...",,"Aliments et boissons à base de végétaux, Alime...",vegetables,28.000000,1.100000,0.200000,4.100000


In [21]:
import requests

In [46]:
r = search_image('Cake aux légumes')

In [47]:
r

['https://cac.img.pmdstatic.net/fit/http.3A.2F.2Fprd2-bone-image.2Es3-website-eu-west-1.2Eamazonaws.2Ecom.2Fcac.2F2018.2F09.2F25.2Fb48d4c3f-8d09-4cde-8761-b7f739a23300.2Ejpeg/750x562/quality/80/crop-from/center/cake-aux-legumes-d-ete-et-fromage-de-chevre.jpeg']

In [25]:
import json

{'_type': 'Images',
 'instrumentation': {'_type': 'ResponseInstrumentation'},
 'readLink': 'https://api.cognitive.microsoft.com/api/v7/images/search?q=test',
 'webSearchUrl': 'https://www.bing.com/images/search?q=test&FORM=OIIARP',
 'queryContext': {'originalQuery': 'test',
  'alterationDisplayQuery': 'test',
  'alterationOverrideQuery': '+test',
  'alterationMethod': 'AM_JustChangeIt',
  'alterationType': 'CombinedAlterationsChained'},
 'totalEstimatedMatches': 835,
 'nextOffset': 1,
 'currentOffset': 0,
 'value': [{'webSearchUrl': 'https://www.bing.com/images/search?view=detailv2&FORM=OIIRPO&q=test&id=9AE608EF8488887A0BE0157E4479858C84FDC669&simid=608018355506665184',
   'name': 'New antibody test can tell you if you have been infected ...',
   'thumbnailUrl': 'https://tse2.mm.bing.net/th?id=OIP.oZ05ftOllQ0vNjoyyaWKfwHaEJ&pid=Api',
   'datePublished': '2020-03-25T22:39:00.0000000Z',
   'isFamilyFriendly': True,
   'contentUrl': 'https://news.images.itv.com/image/file/2164058/img.jpg'

'{"_type": "Images", "instrumentation": {"_type": "ResponseInstrumentation"}, "readLink": "https:\\/\\/api.cognitive.microsoft.com\\/api\\/v7\\/images\\/search?q=test", "webSearchUrl": "https:\\/\\/www.bing.com\\/images\\/search?q=test&FORM=OIIARP", "queryContext": {"originalQuery": "test", "alterationDisplayQuery": "test", "alterationOverrideQuery": "+test", "alterationMethod": "AM_JustChangeIt", "alterationType": "CombinedAlterationsChained"}, "totalEstimatedMatches": 835, "nextOffset": 1, "currentOffset": 0, "value": [{"webSearchUrl": "https:\\/\\/www.bing.com\\/images\\/search?view=detailv2&FORM=OIIRPO&q=test&id=9AE608EF8488887A0BE0157E4479858C84FDC669&simid=608018355506665184", "name": "New antibody test can tell you if you have been infected ...", "thumbnailUrl": "https:\\/\\/tse2.mm.bing.net\\/th?id=OIP.oZ05ftOllQ0vNjoyyaWKfwHaEJ&pid=Api", "datePublished": "2020-03-25T22:39:00.0000000Z", "isFamilyFriendly": true, "contentUrl": "https:\\/\\/news.images.itv.com\\/image\\/file\\/21

In [8]:
df = df[df['product_name'].str.lower().str.contains('banan')]

In [9]:
df.to_sql('bananas',engine,if_exists='append',index=False)

inference

In [11]:
from food.tools import *
from food.paths import *
from food.psql import *

import pandas as pd
import numpy as np
from food.clipmodel import *
import os
os.environ["HF_DATASETS_OFFLINE"] = "2"
os.environ["TRANSFORMERS_OFFLINE"] = "2"
os.environ["CUDA_VISIBLE_DEVICES"] = "2"
from food.tools import *
from food.psql import *
from food.paths import *
from PIL import Image
from food.clipmodel import image2clip
import custom_pandas as cpd
from tqdm import tqdm


inference product name only

In [14]:
iteration = 1

In [15]:

query = f"""select id,product_name,keywords,ingredients_text,categories,food_groups
             from bananas
            where clip{iteration} is Null """

total = engine.execute(f'select count(*) from ({query}) a').one()

bs = 1
pd_iter = cpd.read_sql_query(query, engine, chunksize=bs, index_col='id')


In [ ]:

for inp in tqdm(pd_iter, desc="clip food inference", total=total[0] // bs):  
    try:
        text =inp.fillna("")
        for c in text.columns: text[c] = text[c].str.replace('NaN','')
        text = text['product_name']+ '. ' + text['food_groups']+ '. '+ text['categories']+ '. ' + text['ingredients_text']+'. ' + text['keywords'] 
        clip = text2clip(text.tolist()[0][:150]).numpy().tolist()
        inp['clip'] = [clip]
        insert_ignore(inp,'foods_big',update=True,update_cols=['clip'],unique_cols=['product_name'])
    except RuntimeError as e:
        print(e)

In [17]:
[df[c]+'. ' for c in df.columns]

UFuncTypeError: ufunc 'add' did not contain a loop with signature matching types (dtype('int64'), dtype('<U2')) -> None

In [16]:
df

,id,product_name,keywords,ingredients_text,categories,food_groups,energy_kcal_100g,proteins_100g,fat_100g,carbohydrates_100g
38,664434,"Ethiquable Bananen Chips Scharf (2,34 Eur / 10...","34, bananen, scharf, 100, eur, chip, ethiquable",gswaarde per 100 g /Valen g/Durchschnittliche ...,NaN,NaN,529.0,2.2,31.0,60.0
74,665066,Smoothie Erdbeer-Banane-Traube,"no-added-sugar, away, erdbeer-banane-traube, t...","40% Erdbeermark , 26% Bananmark , 25% Traubens...","Getränke, Ungezuckerte Getränke",NaN,60.0,0.8,0.2,12.1
297,668761,3Bears Porridge Mohnige Banane,"und, porridge, in, vegan, getreide, lebensmitt...","61,5% _Vollkorn-Haferflocken_, 22% getrocknete...","Pflanzliche Lebensmittel und Getränke, Pflanzl...",breakfast-cereals,370.0,11.0,6.9,62.0
400,670314,LAC Fruchtjoghurt Banane,"milch, gluten-free, fruchtjoghurt, lac, joghur...","Lactosefreiero Joghurt, 18 % Bananen-Zubereitu...","Milchprodukte, Fermentierte Lebensmittel, Ferm...",milk-and-yogurt,104.0,4.3,2.9,14.5
430,670905,FruchtBar Granatapfel-Erdbeere-Pfirsich-Banane...,"smoothie, getränke, und, bio, ohne, granatapfe...","pürierter Pfirsich (60%), pürierte Erdbeere (1...","Pflanzliche Lebensmittel und Getränke, Getränk...",NaN,54.0,0.9,0.5,11.0
...,...,...,...,...,...,...,...,...,...,...
1110394,649319,Kirsche Banane,"zertifiziert, teebeutel, mit, heißgetränke, te...","Äpfel, Hibiskus, Hagebutten, Orangenschalen, A...","Pflanzliche Lebensmittel und Getränke, Getränk...",unknown,2.0,0.0,0.0,0.6
1110670,654088,Banane - Apfel,"riegel, bio, süßer, alnatura, banane, imbis, a...","Bananenflocken* 31%, Apfelsaftkonzentrat* 30%,...","Imbiss, Süßer Snack, Riegel",biscuits-and-cakes,368.0,5.0,8.6,63.9
1110785,656040,Nektar bananowy,"nektar, soki, auchan, napoje, roślin, owocowe,...","Woda, przecier bananowy, cukier, regulator kwa...","Żywność i napoje na bazie roślin, Napoje, Napo...",sweetened-beverages,45.0,0.5,0.5,10.0
1111184,662304,Lycka Banane Avocado Haselnuss,"gmbh, banane, süßstoff, kalorienarme, eistee, ...","Wasser, Avocadosoße* (15%) (Agavendicksaft*, A...","Getränke, Getränke mit Süßstoff, Kalorienarme ...",teas-and-herbal-teas-and-coffees,126.0,1.3,6.2,14.0
